In [3]:
import pandas as pd
import numpy as np
%matplotlib inline 

In [4]:
# 2014 
df = pd.read_csv(r'I:\T2040\soundcast_2014\outputs\daysim\_trip.tsv', sep='\t')
hh = pd.read_csv(r'I:\T2040\soundcast_2014\outputs\daysim\_household.tsv', sep='\t')
parcels = pd.read_csv(r'I:\T2040\soundcast_2014\inputs\accessibility\parcels_urbansim.txt', sep = ' ')
parcel_housing = pd.read_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\parcel_housing_costs_2014_lodes_tracts.csv') 

In [5]:
# AAA Calculations availavble for small, medium, large Sedans (and average of these), SUV, Minivan

# Analysis of survey could be used to allocate vehicle type distributions?

# http://publicaffairsresources.aaa.biz/wp-content/uploads/2016/03/2016-YDC-Brochure.pdf
# results are averages for sedans

# Annualalize costs as 262
annual_factor = 262

# Costs by miles
# Operating costs total (gas + maintenance + tires) using model estimates for this in the travcost field
depreciation_per_mile = 3759.0/15000    # depreciation by mile

# Annual costs
insurance = 1222    #(full-coverage insurance, license, registration, taxes, depreciation @ 15,000 miles per year), finance charge
taxes = 687    # license, registration, taxes
finance = 683    # load financing

annual_fixed_costs = insurance+taxes+finance

In [6]:
# to calculate activity durations for parking cost:
day_minutes = 1440
minutes_hr = 60.0
cents_dollar = 100

In [7]:
# get total travel cost (based on vmt) for household records
hh_mile_costs = df.groupby('hhno').sum()[['travcost']]
hh_mile_costs = hh_mile_costs.reset_index()

hh = pd.merge(hh, hh_mile_costs, on='hhno', how='left')

# Some households have 0 travel cost
hh['travcost'] = hh['travcost'].fillna(0)

# Total vehicle miles traveled per household to calculate mile-based costs
driver_trips = df[df['dorp'] == 1]

# Get total travel distance 
hh_travdist = df.groupby('hhno').sum()[['travdist']]
hh_travdist = hh_travdist.reset_index()

hh = pd.merge(hh, hh_travdist, on='hhno', how='left')

# annual operating costs
driver_trips['operating_cost_drivers'] = driver_trips['travcost']*annual_factor
driver_trips['travdist'] = driver_trips['travdist'].fillna(0)
driver_trips['depreciation'] = driver_trips['travdist']*depreciation_per_mile*annual_factor

#calculating parking costs based on activity duration
# the results look weird - add in parking cost later
driver_trips = pd.merge(driver_trips, parcels[['PARCELID','PPRICHRP']], left_on='dpcl', right_on = 'PARCELID')
# if a trip spans midnight you need to adjust across the day - like this:
driver_trips['duration'] = np.where(driver_trips['endacttm']<driver_trips['arrtm'], day_minutes-driver_trips['arrtm'] +driver_trips['endacttm'], driver_trips['endacttm']-driver_trips['arrtm'])/minutes_hr
driver_trips['parking_cost'] = (driver_trips['duration']*driver_trips['PPRICHRP']*annual_factor)/cents_dollar



driver_trips_hh = driver_trips.groupby('hhno').sum()[['operating_cost_drivers','depreciation','parking_cost']].reset_index()
hh = pd.merge(hh, driver_trips_hh, on = 'hhno')

# fixed costs as function of number of vehicles owned
hh['veh_insurance'] = hh['hhvehs']*insurance
hh['veh_taxes'] = hh['hhvehs']*taxes
hh['veh_finance'] = hh['hhvehs']*finance

hh['annual_auto_costs'] = hh['operating_cost_drivers']+hh['depreciation']+hh['veh_insurance']+hh['veh_taxes']+hh['veh_finance']
hh['annual_auto_costs']=hh['annual_auto_costs'].fillna(0)
# Need to add per mile depreciation costs

# Calculate transit cost
transit_trips = df[df['mode'] == 6]

hh_travdist_transit_cost = transit_trips.groupby('hhno').sum()[['travcost']]
hh_travdist_transit_cost['transit_cost'] = hh_travdist_transit_cost['travcost']
hh_travdist_transit_cost = hh_travdist_transit_cost.reset_index()
hh_travdist_transit_cost['annual_transit_cost'] = hh_travdist_transit_cost['transit_cost']*annual_factor

hh = pd.merge(hh, hh_travdist_transit_cost[['hhno','annual_transit_cost']], on='hhno', how='left')
hh['annual_transit_cost'] = hh['annual_transit_cost'].fillna(0)

hh['total_cost']= hh['annual_transit_cost']+hh['annual_auto_costs']
hh = hh.loc[hh['hhincome']>hh['total_cost']]
hh['percent_transport_cost'] = hh['total_cost']/hh['hhincome']
hh['percent_transport_cost'] =hh['percent_transport_cost'].fillna(0)

print hh['total_cost'].mean()
print hh['veh_insurance'].mean()
print hh['veh_taxes'].mean()
print hh['operating_cost_drivers'].mean()
print hh['depreciation'].mean()
print hh['annual_transit_cost'].mean()
print hh['parking_cost'].mean()
print hh['percent_transport_cost'].mean()

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

11030.5148324
2473.99768732
1390.86449361
2577.46964864
3129.28900109
76.1277002655
442.390041972
0.175471651091


In [8]:
#zone_tractid = pd.merge(zone_tract,tractids, left_on= 'GEOID', right_on = 'GEOID10')
#hh_tract = pd.merge(hh,zone_tractid, left_on = 'hhtaz', right_on= 'taz_p').groupby('GEOID10').mean()

In [9]:
#hh_tract.to_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\hh_costs_tract.csv')

In [10]:
parcel_housing.columns

Index([u'aparks', u'empedu_p', u'empfoo_p', u'empgov_p', u'empind_p',
       u'empmed_p', u'empofc_p', u'empoth_p', u'empret_p', u'emprsc_p',
       u'empsvc_p', u'emptot_p', u'hh_p', u'lutype_p', u'nparks', u'parcelid',
       u'parkdy_p', u'parkhr_p', u'ppricdyp', u'pprichrp', u'sqft_p',
       u'stugrd_p', u'stuhgh_p', u'stuuni_p', u'taz_p', u'xcoord_p',
       u'ycoord_p', u'lat_lon_geog', u'geometry', u'index_right', u'final_bg',
       u'blkgrp', u'blkgrps', u'population', u'households', u'land_acres',
       u'ht_ami', u'ht_80ami', u'ht_nmi', u'h_ami', u'h_80ami', u'h_nmi',
       u't_ami', u't_80ami', u't_nmi', u'co2_per_hh_local',
       u'co2_per_acre_local', u'autos_per_hh_ami', u'autos_per_hh_80ami',
       u'autos_per_hh_nmi', u'vmt_per_hh_ami', u'vmt_per_hh_80ami',
       u'vmt_per_hh_nmi', u'pct_transit_commuters_ami',
       u'pct_transit_commuters_80ami', u'pct_transit_commuters_nmi',
       u't_cost_ami', u't_cost_80ami', u't_cost_nmi',
       u'auto_ownership_cost_am

In [11]:
hh_tracts = pd.merge(hh, parcel_housing, left_on = 'hhparcel', right_on = 'parcelid')

In [17]:
hh_tracts.head().to_clipboard()

In [18]:
hh_averages = hh_tracts.groupby('final_bg').mean()

In [20]:
hh_averages.to_csv(r'R:\SoundCastDocuments\metrics\housing_transport_costs\transport_cost_by_tract2.csv')